<a href="https://colab.research.google.com/github/rajpabari/gflownets-rlhf/blob/main/Generate_label_pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!git clone https://github.com/rajpabari/gflownets-rlhf/
import pandas as pd
import numpy as np
from itertools import combinations

fatal: destination path 'gflownets-rlhf' already exists and is not an empty directory.


In [36]:
train_data=pd.read_csv("gflownets-rlhf/data/train.csv")
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [37]:
def clc(dataframe, weights, columns, new_column_name):
    # Calculate the linear combination
    linear_combination = np.dot(dataframe[columns], weights)
    
    # Add the new column to the DataFrame
    dataframe[new_column_name] = linear_combination
    
    return dataframe
train_data = clc(train_data,[0.5,1,0.5,1,0.5,1],train_data.columns[2:],"score")
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,score
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0.0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0.0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0.0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0.0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0.0


In [38]:
train_data.index

RangeIndex(start=0, stop=159571, step=1)

In [39]:
from tqdm import tqdm
import itertools
import random
def create_pairwise_dataset(df,max_size=250000):
    # Create all pairwise combinations of comments
    pool = np.random.permutation(df.index.stop)
    pool2 = np.random.permutation(df.index.stop)
    # Initialize lists to store the data
    pairs = []
    labels = []
    print(pool.shape,df.index.stop)
    # Iterate through the combinations
    k=0
    with tqdm(total=max_size) as pbar:
      while len(pairs) < max_size:
          k+=1
          if k == df.index.stop-1:
              pool = np.random.permutation(df.index.stop)
              pool2 = np.random.permutation(df.index.stop)
              k=0
          #print(k,pool[k],pool2[k])
          i=pool[k]
          j=pool2[k]
          comment_i = df.loc[i, 'comment_text']
          comment_j = df.loc[j, 'comment_text']
          score_i = df.loc[i, 'score']
          score_j = df.loc[j, 'score']
          
          # Determine the label based on the scores
          if score_i < score_j:
              label = 1
          elif score_i > score_j:
              label = 0
          else:
            continue
          pairs.append((comment_i, comment_j, label))
          pbar.update(1)
    
    return  pd.DataFrame(pairs, columns=["comment1","comment2","score"])
train_df = create_pairwise_dataset(train_data)
train_df.head()

(159571,) 159571


100%|██████████| 250000/250000 [00:53<00:00, 4716.64it/s]


,comment1,comment2,score
0,Are you really that stupid? \n\nDo you know th...,start|importance= low|attention= |peer-review=...,0
1,"""\n\n With what software did you create the pi...",Let me break it down for ya'll. Murdering inno...,1
2,"Monitor! Are these technology-related nouns, o...","First of all, I've never been blocked before.\...",1
3,"I'm lifting a glass, myself. Voting just ended...",JAYJG HAS ALREADY EATEN HIS TABLET! YOU MUST D...,1
4,you fucking piece of shit,"""\n\nWhen you say, """"The content of the articl...",0


In [40]:
train_df.to_csv("./train_reward.csv")

In [41]:
import gc
gc.collect()

0

In [34]:
test_data=pd.read_csv("gflownets-rlhf/data/test.csv")
test_labels=pd.read_csv("gflownets-rlhf/data/test_labels.csv")
train_data.head(), test_labels.head()

(                 id                                       comment_text  toxic  \
 0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
 1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
 2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
 3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
 4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   
 
    severe_toxic  obscene  threat  insult  identity_hate  score  
 0             0        0       0       0              0    0.0  
 1             0        0       0       0              0    0.0  
 2             0        0       0       0              0    0.0  
 3             0        0       0       0              0    0.0  
 4             0        0       0       0              0    0.0  ,
                  id  toxic  severe_toxic  obscene  threat  insult  \
 0  00001cee341fdb12     -1            